<a href="https://colab.research.google.com/github/jeong1suk/Capstone_ElectricCar/blob/wonsuk/findplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install timm

In [ ]:
# !pip uninstall torch torchvision
!pip install torch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2

In [ ]:
import torch
print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

In [ ]:
torch.cuda.is_available()

In [ ]:
import torch
import timm
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, datasets
import os
import glob
from tqdm import tqdm
from PIL import Image
import random
from sklearn.model_selection import train_test_split
import torch
import cv2
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
pretrained_resnet=timm.create_model('resnet18', pretrained=True, num_classes=0, global_pool='')

In [ ]:
class char_mymodel(nn.Module):
    def __init__(self, pretrained_model):
        super(char_mymodel, self).__init__()
        self.pretrained = pretrained_model
        
        self.additional_layers1 = nn.Sequential(
            nn.Conv2d(512, 45, 3, 1, 1),
            nn.BatchNorm2d(45),
            nn.ReLU(),
        )
        self.additional_layers2 = nn.Sequential(
            nn.AdaptiveAvgPool2d(output_size=(1,7))
        )
        
    def forward(self, x):
        out = self.pretrained(x)
        out = self.additional_layers1(out)
        out = out[:,:,:,0:7]
        out = self.additional_layers2(out)
        out = out.permute(0, 3, 1, 2)
        return out 

In [ ]:
model = char_mymodel(pretrained_model=pretrained_resnet)

In [ ]:
print(model)

In [ ]:
class load_data(Dataset):
    def __init__(self, path, transform=None):
        self.img_list = glob.glob(path) # 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환
        self.transform = transform

    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
        img_name = self.img_list[idx]
        ret_img = cv2.imread(img_name)
        #print(self.img_list[idx])
        return ret_img

In [ ]:
def visualization(pred: torch.Tensor, gt:torch.Tensor):
    label_template = ['가', '나', '다', '라', '마', '거', '너', '더', '러', '머',
                      '버', '서', '어', '저', '고', '노', '도', '로', '모', '보',
                      '소', '오', '조', '구', '누', '두', '루', '무', '부', '수',
                      '우', '주', '허', '하', '호', '0', '1', '2', '3', '4',
                      '5', '6', '7', '8', '9']

    pred = pred.type(torch.int64).cpu().detach().numpy()
    gt = gt.type(torch.int64).cpu().detach().numpy()

    gt_str = ''
    for a in range(7):
        gt_str += label_template[gt[0][a]]
    print("ground truth: " + gt_str)

    pred_str = ''
    for a in range(7):
        pred_str += label_template[pred[0][a]]
    print("prediction: " + pred_str)

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
train_dataset = load_data("/content/drive/MyDrive/licenseplate/train/*.jpeg")
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

In [ ]:
print(len(train_dataset))

In [ ]:
for sample_batch in train_loader:
    print(sample_batch)
    #print(sample_batch[0].shape, sample_batch[1].shape)
    break

In [ ]:
epoch = 256
device = torch.device('cuda')
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

x = torch.randn(64, 3, 128, 256, requires_grad=True).to(device)

In [ ]:
best = 0
count = 0

for ep in range(epoch):
    print("epoch {}/{}".format(ep, epoch-1))
    print('-'*10)
    avg_loss = AverageMeter()
    
    model.train()
    for idx, img in tqdm(enumerate(train_loader), total=len(train_loader)):
        data_x = img.type(torch.float32).to(device)
        #print("data_x: ", data_x.shape)
        data_x = data_x.permute(0,3,1,2)
        out = model(data_x)
        out = out.squeeze(-1)
        #print("shape: ", out.shape)